In [1]:
import pandas as pd
import numpy as np
import re
import os
os.listdir('./kids_data')

df_member = pd.read_excel(
    './kids_data/Member_data.xlsx',
    dtype={
        '고객번호': str,
        '성별': str,
        '결혼유무': str,
        '거주지역': str,
        '연령': float,   # 연령도 혹시 결측이 있으면 float로
        '결제등록카드': str,
        '유입경로': str,
        '자녀여부': str
    }
)
df_product = pd.read_csv('./kids_data/Product_data.csv')
df_sales = pd.read_csv('./kids_data/Sales_data.csv', encoding='utf-8', index_col=0
           ,dtype={
        '주문번호': str,
        '배송완료일': str,
        '배송시작일': str,
        '구매일': str,
        '구매시월령': float,   # 연령도 혹시 결측이 있으면 float로
        '고객번호': str,
        '구매금액': float,
        '결제금액': float,
        '물품대분류': str,
        '상품명': str       
    }
)


merged_df = pd.merge(df_sales, df_member, how='left', left_on = ['고객번호'], right_on=['고객번호'])
df = pd.merge(merged_df, df_product, how='left', left_on = ['상품명'], right_on=['상품명'])
print(df.shape)

del merged_df
del df_member
del df_sales
del df_product

# 물품대분류_x	 > 물품대분류_y 정확도 높음
df.drop(columns=['물품대분류_y'], axis=1, inplace=True)

df.columns = ['주문번호', '배송완료일', '배송시작일', '구매일', '구매시월령', '고객번호', '구매금액', '결제금액',
       '물품대분류', '상품명', '성별', '결혼유무', '거주지역', '연령', '결제등록카드', '유입경로', '자녀여부',
       '물품판매량', '제품단가']
df.dropna(subset=['성별'], inplace=True)
df.isna().sum()

(210039, 20)


주문번호          0
배송완료일         0
배송시작일         0
구매일           0
구매시월령     32812
고객번호          0
구매금액          0
결제금액          0
물품대분류         0
상품명           0
성별            0
결혼유무          0
거주지역          0
연령            0
결제등록카드        0
유입경로          0
자녀여부          0
물품판매량         0
제품단가          0
dtype: int64

In [2]:
keywords = {

    '티엔 AC 매끈 진정 시트 마스크(7매)':'SKC',
    '티엔 BT21 AC 진정 시트 마스크(7매)':'SKC',
    '스킨카밍 엔자임 멀티토닉':'SKC',
    '액티브 엔자임 볼륨 앤 스칼프 샴푸':'SKC',
    '손 소독제 겔':'FMC',
    '마일드 페미닌 워시':'FC',

    '(증정) 티엔 매끈 진정 마스크팩 (EA)':'SKC',
    '트레제메 모이스처 리치 린스 900ml X 2':'SKC',
    '트레제메 모이스처 리치 샴푸 900ml X 2':'SKC',
    '[임직원] 티엔 BT21 AC 컨트롤 세트 (토너+로션+버블폼 3종)':'SKC',
    
    '그린핑거 손소독 물티슈 캡형':'FMC',
    
    '레고':'기타',
    
    '체험':'체험팩',
    '맘큐 허그박스':'체험팩',
    '하기스 샘플러 3단계':'체험팩',
    '하기스 샘플러 4단계':'체험팩',
    
    '라네이처 페미닌 티슈':'FC',
    '좋은느낌 라인프렌즈 유기농 순면':'FC',
    '좋은느낌 라인프렌즈 유기농순면':'FC',
    
    'KF94':'FMC',
    '크리넥스 메이브리즈 황사마스크':'FMC',
    '크리넥스 도톰한 순수 3겹 35m-36롤':'FMC',
    '스카트 세트':'FMC',
    '진정 시트 마스크':'FMC',
    
    '물티슈 캡형':'물티슈',
    '아기물티슈':'물티슈',
    '하기스 수딩케어 물티슈':'물티슈',
    '물티슈 세트':'물티슈',
    '하기스 뉴본케어 물티슈':'물티슈',
    '하기스 모이스처케어 물티슈':'물티슈',
    '하기스 네이처메이드 밤부 물티슈 휴대용 20매X12팩':'물티슈',
    
    '티엔 AC 뽀송 피지조절 시트 마스크(7매)' :'SKC',
    '티엔 AC 매끈 진정 시트 마스크(7매)':'SKC',
    '티엔 BT21 AC 진정 시트 마스크(7매)':'SKC',
    '클렌징 티슈':'SKC',
    '클렌징 폼':'SKC',
    '티엔 BT21 촉촉 워터 클렌징 티슈':'SKC',
    '그린핑거 야외놀이 선크림':'SKC',
    '대용량 베이비밤':'SKC',
    '그린핑거 마이키즈':'SKC',
    '그린핑거 베이비':'SKC',
    '퓨어클렌징 티슈':'SKC',
    '선팩트':'SKC',
    '레드투래드 안티더스트 클렌징 오일':'SKC',
    '레드투래드 안티더스트 배리어 미스트':'SKC',

    '하기스 에어솔솔 썸머기저귀':'기저귀',
    
    '티슈':'FMC', # 로션 때문에 먼저 실행
    '행주':'FMC',
    '수세미':'FMC',
    '청소포':'FMC',
    '화장지':'FMC',
    '키친타월':'FMC',
    '키친타올':'FMC', 
    '마이비데':'FMC',
    '막대걸레':'FMC',
    '습기제거제':'FMC',
    '주방 위생 세트':'FMC',
    '콧물전용 물티슈':'FMC',
    '크리넥스 클래식':'FMC',    
    '크리넥스 순수 3겹':'FMC',
    '크리넥스 울트라클린':'FMC',
    '크리넥스 실크소프트':'FMC',
    '크리넥스 NEW 디럭스':'FMC',
    '크리넥스/스카트 세트':'FMC',
    '크리넥스 3겹 클린케어':'FMC',
    '크리넥스 3겹 천연펄프':'FMC',
    '크리넥스 + 스카트 세트':'FMC',
    '크리넥스 수프림 소프트':'FMC',
    '크리넥스 디럭스 신수성':'FMC',
    '크리넥스 보습에센스로션':'FMC',
    '크리넥스 호호바수딩로션':'FMC',
    '크리넥스 3겹 울트라클린':'FMC', 
    '크리넥스 3겹 순수 소프트':'FMC',
    '크리넥스 수앤수 코튼클린':'FMC',
    '크리넥스 3겹 데코앤소프트':'FMC',
    '크리넥스 3겹 수프림 소프트':'FMC',
    '크리넥스 데코앤소프트 클래식':'FMC',
    '크리넥스 여행용 호호바수딩로션':'FMC',
    '스타일마스크':'FMC',
    '크리넥스 황사마스크 뉴디자인플러스':'FMC',
    '크리넥스 가습촉촉 마스크':'FMC',
    '입체 마스크':'FMC',
    '크리넥스 황사용 마스크':'FMC',
    '크리넥스 데일리 입체마스크':'FMC',
    '크리넥스 카카오 프렌즈 건강 마스크':'FMC',

    '더블하트':'더블하트',
    '안심 위생 관리 세트':'더블하트',    
    '이유식기 준비 끝! 세트':'더블하트',
    '젖병 깔끔 세척 끝! 세트':'더블하트',
    '젖병트윈팩':'더블하트',

    '택배파손':'택배파손',

    '디펜드 스타일 언더웨어':'SrC',
    '디펜드 안심플러스 언더웨어':'SrC',
    '디펜드 안심케어 매직테이프':'SrC',
    '디펜드 스타일 라이너 여성용 무향 롱':'SrC',

    '보행기':'기타',
    '레드투래드':'기타',
    '쥬트 린넨 쇼퍼백':'기타',
    '스카트 이벤트 상품':'기타',
    '키즈밴드 랜덤 1종 증정':'기타',
    '미미루 뱀부 이중 블랭킷':'기타',
    '낮잠이불세트':'기타',
    '롤매트':'기타',
    '너프건':'기타',
    '헤이즐넛 코코아 스프레드':'기타',
    '식탁의자':'기타',
    '비타민':'기타',
    '젤리':'기타',
    '방수요':'기타',
    '배냇슈트':'기타',
    '아기띠침받이':'기타',
    '외출용 필수 3종세트':'기타',
    '유아 대형 목욕타올':'기타',
    '손타올':'기타',
    '유아 외출용 미니타올3장':'기타',
    '속싸개':'기타',
    '10종 기획 특선 출산 세트':'기타',
    '배냇저고리':'기타',
    '아기띠 침받이':'기타',
    '출산준비세트':'기타',
    '아기띠워머':'기타',
    '유아목욕타올':'기타',
    '턱받이':'기타',
    '드시모네':'기타',
    '디어에코':'기타',
    '디즈니베이비':'기타',
    '라그랑':'기타',
    '라밀로우':'기타',
    '라인프렌즈':'기타',
    '레고':'기타',
    '룰라러브':'기타',
    '리틀누':'기타',
    '리틀마마':'기타',
    '릴라스틸':'기타',
    '마이포코리아':'기타',
    '맘마밀':'기타',
    '맘스앱솔루트':'기타',
    '매일우유':'기타',
    '매직캔':'기타',
    '먼치킨':'기타',
    '메가블럭':'기타',
    '메릴린':'기타',
    '몽슈레':'기타',
    '무무브':'기타',
    '믹스드페어스':'기타',
    '바오밥성장그림':'기타',
    '베이비머핀':'기타',
    '베이비아인슈타인':'기타',
    '보니타베베':'기타',
    '보보앤부':'기타',
    '보쉬토이':'기타',
    '본토이':'기타',
    '브라이트스타트':'기타',
    '브리카':'기타',
    '비더마틱':'기타',
    '비박스':'기타',
    '비츠':'기타',
    '뽀드미엘':'기타',
    '뿌띠슈':'기타',
    '쁘띠베베':'기타',
    '산골간식':'기타',
    '상하목장':'기타',
    '상하치즈':'기타',
    '순수담다':'기타',
    '스와들디자인':'기타',
    '스와들업':'기타',
    '시크릿쥬쥬':'기타',
    '신비아파트':'기타',
    '실리만':'기타',
    '씨알로':'기타',
    '아이스크림페어리':'기타',
    '아이엠스페셜':'기타',
    '아이헤이트먼데이':'기타',
    '야마토야':'기타',
    '에르고베이비':'기타',
    '에르고파우치':'기타',
    '에불로보':'기타',
    '엔픽스':'기타',
    '엘비니':'기타',
    '엘트라':'기타',
    '오볼':'기타',
    '올라베이비':'기타',
    '요미요미':'기타',
    '윈펀':'기타',
    '유한양행':'기타',
    '율그란':'기타',
    '임직원':'기타',
    '젤리맘':'기타',
    '조이':'기타',
    '하기스 리미티드 비치볼':'기타',
    '카가앤푸치':'기타',
    '캐스돈':'기타',
    '캐시크루즈':'기타',
    '콱스':'기타',
    '크로텍':'기타',
    '키썸플레이':'기타',
    '킨더초콜릿':'기타',
    '탑브라이트':'기타',
    '테스':'기타',
    '트루자임':'기타',
    '페레로로쉐':'기타',
    '퓨리티':'기타',
    '플레이도':'기타',
    '피에스타':'기타',
    '필리앤임프스':'기타',
    '핑크퐁':'기타',
    '해밀턴':'기타',
    '미라클':'기타',
    '메릴린':'기타',
    '손소독 물티슈':'기타',
    'NEW스턴트레이서 오렌지':'기타',
    '립톤 아이스티 피치':'기타',
    '스페셜기프트SET':'기타',
    '애착인형':'기타',
    '캐스돈':'기타',
    '트레제메':'기타',
    '티엔 하이드로콜로이드밴드':'기타',
    '아쿠아슈즈':'기타',
    '쿨토시':'기타',

    '하기스 매직핏 꿀벅지 아기용 팬티':'팬티',
    '하기스 맥스드라이 팬티':'팬티',
    '하기스 NEW 네이처메이드 썸머팬티':'팬티',
    '하기스 에어솔솔 팬티':'팬티',
    '하기스 보송보송팬티':'팬티',
    '하기스 매직핏팬티':'팬티',
    '하기스 네이처메이드 팬티':'팬티',
    '하기스 에어솔솔 썸머팬티':'팬티',
    '굿나이트 남아 중형':'팬티',
    '굿나이트 여아 중형':'팬티',
    '굿나이트 여아 대형':'팬티',
    '굿나이트 남아 대형':'팬티',
    '하기스 매직팬티 컴포트핏':'팬티',
    
    '스마트 프리미엄 멤버십 가입선물':'기저귀',
    '하기스 매직 컴포트':'기저귀',
    '스마트 베이비 모니터':'기저귀', # 기저귀에 붙이는 거니까 !
    '하기스 맥스드라이':'기저귀',
    '하기스 네이처메이드':'기저귀',
    '하기스 NEW 네이처메이드':'기저귀',
    '하기스 보송보송':'기저귀',
    '하기스 매직핏 꿀벅지 아기용 기저귀':'기저귀',

    '그린핑거 베이비 세라마이드 크림':'SKC',
    '그린핑거 마이키즈 자두야에디션 샴푸':'SKC',
    '그린핑거 베이비 세라마이드 로션':'SKC',
    '그린핑거 마이키즈 워시 대용량':'SKC',
    '그린핑거 베이비 세라마이드 세트':'SKC',
    '그린핑거 마이키즈 자두야에디션 촉촉 립밤':'SKC',
    '그린핑거 마이키즈 워시 500ml(대용량) + 샴푸 500ml(대용량)':'SKC',
    '선로션':'SKC',
    '대용량 버블스':'SKC',
    '손소독제겔':'SKC',
    '손소독제 겔':'SKC',
    '손 소독제 겔':'SKC',

    '날개':'FC',
    '날개형':'FC',
    '팬티라이너':'FC',
    '오버나이트':'FC',
    '탐폰':'FC',
    '디펜드 맘스':'FC',
    '화이트 시크릿홀 수퍼롱':'FC',
    '라네이처 데일리라이너 롱':'FC',
    '라네이처 날개 오버나이트':'FC',
    '라네이처 세트':'FC',
    '라네이처 페미닌 워시 젤':'FC',
    '라네이처 페미닌 워시 폼':'FC',
    '디펜드 스타일 라이너 여성용':'FC',
    '여성 청결제 페미닌 파우더':'FC',
    '여성 질건강 유산균':'FC',
    '[마더니아]':'FC',
    '라네이처 시그니처 세트':'FC',

    '팬티':'팬티',

    '기저귀':'기저귀',

    '하기스 샘플러':'기저귀',
    '하기스 크린베베 순면감촉':'기저귀',
    '물놀이팬티':'기저귀',

    '티엔 매끈 진정 마스크팩':'기타',
    '그린핑거 촉촉한 자연보습':'SKC',
    '그린핑거 마이키즈':'SKC',
    '그린핑거 힘센보습':'SKC',
    '그린핑거 베이비':'SKC',
    '그린핑거 인텐시브 투명 선밤':'SKC',
    '그린핑거 퓨어 클렌징 워터':'SKC',
    '젤로션':'SKC',
    '로션':'SKC',
    '샴푸&워시':'SKC',
    '크림':'SKC',
    '그린핑거 마이키즈 + 자두야에디션 세트':'SKC',
    '그린핑거 마이키즈 + 힘센보습':'SKC',
    '거품샴푸':'SKC',
    '거품워시':'SKC',
    '샴푸':'SKC',
    '그린핑거 마이키즈 세트':'SKC',
    '얼굴로션':'SKC',
    '워시':'SKC',
    '립밤':'SKC',
    '핸드워시':'SKC',
    '훼이셜솝':'SKC',
    '미네랄 선팩트':'SKC',
    '미네랄팩트':'SKC',
    '그린핑거 베이비 세라마이드 세트':'SKC',
    '훼이셜로션':'SKC',
    '선로션':'SKC',
    '선크림':'SKC',
    '선밤':'SKC',
    '선쿠션':'SKC',
    '선팩트':'SKC',
    '비누':'SKC',
    '울트라크림':'SKC',
    '탑투토워시':'SKC',
    '클렌징 워터':'SKC',
    '퓨어클렌징 티슈 캡':'SKC',
    '선스틱':'SKC',
    '선젤':'SKC',
    '크리미오일':'SKC',
    '티엔 AC 매끈 진정 시트 마스크':'SKC',
    '티엔 촉촉':'SKC',
    '수딩젤로션':'SKC',
    '로션프레쉬':'SKC',
    '바디워시':'SKC',
    '티엔 BT21':'SKC',
    '토너':'SKC',
    '바세린 퓨어스킨 젤리':'SKC',
    '판테딘 수딩젤':'SKC',
    '판테딘 탑투토워시':'SKC',
    '클렌징':'SKC',
    '미스트':'SKC',
    '도브 화이트 뷰티바':'SKC',

    '릴리유 퓨어 무민 스프링에디션 캡형':'물티슈',
    '물티슈':'물티슈', # 콧물 전용 물티슈 때문에 나중에
    
    '마스크':'FMC', # 마스크 팩 때문에 나중에
}

from collections import OrderedDict
keywords=OrderedDict(keywords)
for kw,cat in keywords.items(): 
    df.loc[df['물품대분류'].isna() & df['상품명'].str.contains(kw,na=False,regex=False),'물품대분류']=cat


In [ ]:
# 3. sales 전처리
df['배송완료일'] = pd.to_datetime(df['배송완료일'])
df['배송시작일'] = pd.to_datetime(df['배송시작일'])

df['구매일'] = pd.to_datetime(df['구매일'])
df['주문일로부터배송완료시간'] = (df['배송완료일'] - df['구매일']).dt.days
df['택배이동기간'] = (df['배송완료일'] - df['배송시작일']).dt.days

# 4. 파생변수 생성
def 기저귀(x):
    상품명 = x['상품명']
    제외어 = ['젖꼭지', '칫솔', '브러쉬', '쌀과자']
    if any(단어 in 상품명 for 단어 in 제외어): return np.nan
    if '1단계' in 상품명: return '1단계 : 태어난지 한달 무렵까지'
    elif '2단계' in 상품명: return '2단계 : 한달 무렵부터'
    elif '3단계' in 상품명: return '3단계 : 뒤집고 기는 아기'
    elif '4단계' in 상품명: return '4단계 : 걷고 시작하는 아기'
    elif '5단계' in 상품명: return '5단계 : 아장아장 걷는 아기'
    elif '6단계' in 상품명: return '6단계 : 잘 뛰고 활동적인 아기'
    elif '7단계' in 상품명: return '7단계 : 성장이 빠르고 활동적인 아기'
    elif '8단계' in 상품명: return '8단계 : 성장이 빠르고 아주 활동적인 아기'
    else: return np.nan


def 남아여아공용(x):
    if pd.isna(x): return np.nan  # 결측값 처리
    if '남아' in x: return '남아'
    elif '여아' in x: return '여아'
    elif '공용' in x: return '공용'
    else: return np.nan
        

df['기저귀단계'] = df.apply(기저귀, axis=1)
df['남아여아공용'] = df['상품명'].apply(남아여아공용)


# 브랜드 컬럼 생성
유한킴벌리 = pd.DataFrame(
    {'브랜드':[
        '하기스', '그린핑거', '굿나이트', '크리넥스', '좋은느낌', '화이트', '스카트', '디펜드', '매일우유',
'베베당', '매직캔', '팔도', '동원', '퓨어잇', '유한양행', '프라하우스', '실리만', '먼치킨', '요미요미', '푸고', '코니스', '퓨어락', '캐치티니핑', '브라이트스타트',
'디즈니베이비', '아넬라', '베베스트', '도담도담 오가닉', '두리', '더마틱스', '파스퇴르', '이지캔', '아솝느', '바이엘', '돗투돗',
'핑크퐁', '꿈비', '무무브', '필리엔임프스', '베이비아인슈타인', '대상웰라이프', '오솔로N59', '율그란', '콕시클',
'도티도그', '투티밤비니', '에르고베이비', '오볼', '로지오가닉', '두나', '마이니', '애니멀플래닛', '조이',
'몰랑', '스와들업', '뽀로로', '캥거루루', '종외제약', '프로텍트어베드', '벨몽', '세노비스키즈', '브리카',
'엘트라', '샤미부', '인제뉴어티', '클레멘토니', '리안', '21세기킥보드', '유니끄', '소다스트림', '비츠',
'세노비스', '피에스타', '마더니아', '시크릿쥬쥬', '라그랑', '미니언즈', '산골간식', '포켓몬', '오리벨',
'나스파', '바큐바', '신비아파트', '안국건강', '아이리버', '보보엔부', '몽슈레', '뉴나', '킨더고',
'올라베이비', '조지루시', '베이비버스', '흔한남매', '선스태쉬', '에불로보', '라밀로우', '디어에코', '콱스',
'보니타베베', '나니아', '룰라러브', '윈펀', '뉴오리진', '테스', '플레이그로', '일양약품', '바오밥 성장그림',
'루이', '크로텍', '몬스터토이즈', '코코지', '진인프라', '뿌띠슈', '마뉴엘라', '블랑가또', '산리오',
'마더스베이비', '탁가온', '지노렉스', '미니막스', '코체티', '3스프라우츠', '서울우유', '유아마이', '알파슈',
'이체크', '하카', '엔젤비닷', '제스프리', '압타밀', '롯데칠성음료', '일동후디스', '키즈꼬모', '동국제약','쁘띠베베','라네이처','릴리유','더블하트'
        ,'닥터마밍','맘마밀','데르뜨','도담도담오가닉','맘큐','보쉬토이','누들앤부','캐스돈','티엔','믹스드페어스','메가블럭','아이스크림페어리','야마토야','드시모네','베이비머핀','메릴린','리틀누'
    ,'본토이','보보앤부','뽀드미엘','리틀마마','엘비니','비박스','카가앤푸치','아이헤이트먼데이',' 트레제메','바세린','립톤','도브','엘비니','키썸플레이','트레제메'
    ,'너프','캐시크루즈','순수담다','씨알로','아이엠스페셜','스와들디자인','라인프렌즈'
        ,'레고','필리앤임프스','비더마틱','릴라스틸','퓨리티','플레이도우','상하목장','상하치즈','누텔라','페레로로쉐','맘스앱솔루트'
        ,'트루자임','탑브라이트','엔픽스','마이포코리아','해밀턴','플레이도','네이처메이드','레드투래드','메이브리즈','Go-MINI','에르고파우치','젤리맘'
    ]}
)


def extract_brand(text):
    for brand in 유한킴벌리['브랜드'].values:
        if brand in text:
            return brand
    return 'etc'
    
def remove_brand(text):
    brand = extract_brand(text)
    return text.replace(brand, '').strip() if pd.notna(brand) else text

def 원플원_extract_brand(text):
    if re.search(r'1\s*\+\s*1', text):
        return 1
    return 0

def 핫딜_extract_brand(text):
    if re.search(r'핫딜', text):
        return 1
    return 0


# 브랜드 추출
df['브랜드'] = df['상품명'].apply(extract_brand)

# 상품명에서 브랜드 제거
df['상품명'] = df['상품명'].apply(remove_brand)

# 원플원 컬럼 생성
df['원플원'] = df['상품명'].apply(원플원_extract_brand)

# 핫딜 컬럼 생성
df['핫딜'] = df['상품명'].apply(핫딜_extract_brand)

df['쿠폰적용가강조'] = df['상품명'].str.contains('쿠폰').astype(int)
df['무료배송강조']= df['상품명'].str.contains('무료배송').astype(int)
df['증정강조'] = df['상품명'].str.contains('증정').astype(int).sum()
df['예약구매'] = df['상품명'].str.contains('예약').astype(int).sum()
df['기저귀밴드/팬티'] = df['상품명'].apply(
    lambda x: '팬티형' if '팬티' in x else ('밴드형' if '밴드' in x else '기저귀아님')
)


def 기저귀(x):
    상품명 = x['상품명']
    제외어 = ['젖꼭지', '칫솔', '브러쉬', '쌀과자']
    if any(단어 in 상품명 for 단어 in 제외어): return np.nan
    if '1단계' in 상품명: return '태어난지 한달 무렵까지'
    elif '2단계' in 상품명: return '한달 무렵부터'
    elif '3단계' in 상품명: return '뒤집고 기는 아기'
    elif '4단계' in 상품명: return '걷고 시작하는 아기'
    elif '5단계' in 상품명: return '아장아장 걷는 아기'
    elif '6단계' in 상품명: return '잘 뛰고 활동적인 아기'
    elif '7단계' in 상품명: return '성장이 빠르고 활동적인 아기'
    elif '8단계' in 상품명: return '성장이 빠르고 아주 활동적인 아기'
    else: return np.nan
        
df['기저귀단계'] = df.apply(기저귀, axis=1)

df['아이정보건너뛰기'] = df['구매시월령'].isna().astype(int)

def 자녀월령결측치삽입(x):  # x는 행(row) 하나
    if pd.isna(x['구매시월령']):
        if pd.notna(x['기저귀단계']):
            if x['기저귀단계'] == '걷고 시작하는 아기': return 13
            elif x['기저귀단계'] == '뒤집고 기는 아기': return 8
            elif x['기저귀단계'] == '성장이 빠르고 활동적인 아기': return 29
            elif x['기저귀단계'] == '아장아장 걷는 아기': return 20
            elif x['기저귀단계'] == '태어난지 한달 무렵까지': return 5
            elif x['기저귀단계'] == '잘 뛰고 활동적인 아기': return 24
            elif x['기저귀단계'] == '한달 무렵부터': return 5
    return x['구매시월령']  # 변경 없으면 기존 값 유지

def 자녀월령이상치처리(x):  # x는 행(row) 하나
    if (x['구매시월령']>240):
        if pd.notna(x['기저귀단계']):
            if x['기저귀단계'] == '걷고 시작하는 아기': return 13
            elif x['기저귀단계'] == '뒤집고 기는 아기': return 8
            elif x['기저귀단계'] == '성장이 빠르고 활동적인 아기': return 29
            elif x['기저귀단계'] == '아장아장 걷는 아기': return 20
            elif x['기저귀단계'] == '태어난지 한달 무렵까지': return 5
            elif x['기저귀단계'] == '잘 뛰고 활동적인 아기': return 24
            elif x['기저귀단계'] == '한달 무렵부터': return 5
    else : return x['구매시월령']
    
df['구매시월령'] =df.apply(lambda x: 자녀월령결측치삽입(x), axis=1)
df['구매시월령'] =df.apply(lambda x: 자녀월령이상치처리(x), axis=1)


태아 = 0
초등학생 = 12*8
중학생 = 12*14
고등학생 = 12*17
성인 = 12*20

conditions = [
    df['구매시월령'] >= 성인,
    df['구매시월령'] >= 고등학생,
    df['구매시월령'] >= 중학생,
    df['구매시월령'] >= 초등학생,
    df['구매시월령'] < 태아,
    pd.notna(df['구매시월령'])
]

# 선택값 (조건 개수와 동일하게)
choices = ['성인', '고등학생', '중학생', '초등학생', '태아', '미취학아동']

# np.select의 default는 함수 인자로 전달
df['구매시월령_bin'] = np.select(conditions, choices, default=np.nan)
df.loc[df['구매시월령_bin']=='nan','구매시월령_bin'] = np.nan
# 분포 확인
df['구매시월령_bin'].value_counts(dropna=False) / df.shape[0]

# 2. 전처리
mask = df['상품명'].str.contains('물티슈', na=False)
df.loc[mask, '물품대분류'] = '물티슈'
mask = df['상품명'].str.contains('더블하트', na=False)
df.loc[mask, '물품대분류'] = '기타'
mask = df['상품명'].str.contains('[핑크퐁]', na=False)
df.loc[mask, '물품대분류'] = '기타'
mask = df['상품명'].str.contains('젖병 깔끔 세척 끝! 세트', na=False)
df.loc[mask, '물품대분류'] = '기타'
mask = df['상품명'].str.contains('이유식기 준비 끝! 세트', na=False)
df.loc[mask, '물품대분류'] = '기타'
mask = df['상품명'].str.contains('안심 위생 관리 세트', na=False)
df.loc[mask, '물품대분류'] = '기타'
mask = df['상품명'].str.contains('레드투래드', na=False)
df.loc[mask, '물품대분류'] = 'SKC'

df = df[df['결제금액'] != -35166]


# 우태 코드

df.loc[df['구매시월령'].notna() & (df['자녀여부'] == 'No'), '자녀여부'] = 'Yes'


# 2. 결측치 처리 및 상품명 재정제
df = df.dropna(subset=['상품명']) # 4건 결측치 발생 -> 삭제
df = df.dropna(subset=['구매시월령'])

In [18]:
import re
from pykospacing import Spacing

spacing = Spacing()  # 클래스는 함수 밖에서 한 번만 생성

def clean_product_name(name):
    if not isinstance(name, str):
        return ''
    
    if '허그박스' in name:
        return name

    # 1. 구분자 역할 특수문자 → 공백 치환
    name = re.sub(r'[-_/]', ' ', name)

    # 2. 숫자 제거
    name = re.sub(r'[0-9]', '', name)

    # 3. 영어 제거
    name = re.sub(r'[a-zA-Z]', '', name)

    # 4. 나머지 특수문자 제거 (공백은 유지)
    name = re.sub(r'[{}\[\]!@#$%^&*()=+~`,.<>_]', '', name)

    # 5. 공백 정리
    name = re.sub(r'\s+', ' ', name).strip()

    # 6. 띄어쓰기 보정
    name = spacing(name)

    # 7. 불용어 제거
    stopwords = {
        '건', '매팩', '예약구매', '매', '용', '팩', '봉', '개', '개입', '랜덤발송', '단계',
        '남아', '매팬티형', '여아', '공용', '팬티형', '밴드형',
        '년형', '매밴드형', '입', '여', '캡', '일분', '캡형', '종', '택'
    }
    words = name.split()
    deduped = [w for w in words if w not in stopwords]

    return ' '.join(deduped)

In [ ]:
# 적용
df['상품명_정제'] = df['상품명'].apply(clean_product_name)

In [21]:
# 3. 데이터 타입 변경
for dt_col in '배송완료일 배송시작일 구매일'.split(): 
    df[dt_col] = pd.to_datetime(df[dt_col])

df['주문번호'] = df['주문번호'].astype(object)
df['고객번호'] = df['고객번호'].astype(object)

for cat_col in ['물품대분류','상품명','결혼유무','거주지역','결제등록카드','유입경로','자녀여부','기저귀단계','택배이동기간','주문일로부터배송완료시간','남아여아공용','브랜드','구매시월령_bin']:
    df[cat_col] = pd.Categorical(df[cat_col])

df['결혼유무']= np.where(df['결혼유무']=='미혼',0,1)

# 1. 고객별 총 주문 수 계산
order_count = df.groupby('고객번호')['주문번호'].size().reset_index(name='총주문횟수')
last_orderday = df.groupby('고객번호')['구매일'].max()
# 2. 원본 데이터와 병합
df = df.merge(order_count, on='고객번호', how='left')
df = df.merge(last_orderday, on='고객번호', how='left')

df['결제등록카드'] = df['결제등록카드'].replace('국민카드', '국민은행').replace('신한카드', '신한은행').replace('외환은행', '하나은행')

In [22]:
df.columns

Index(['주문번호', '배송완료일', '배송시작일', '구매일_x', '구매시월령', '고객번호', '구매금액', '결제금액',
       '물품대분류', '상품명', '성별', '결혼유무', '거주지역', '연령', '결제등록카드', '유입경로', '자녀여부',
       '물품판매량', '제품단가', '주문일로부터배송완료시간', '택배이동기간', '기저귀단계', '남아여아공용', '브랜드',
       '원플원', '핫딜', '쿠폰적용가강조', '무료배송강조', '증정강조', '예약구매', '기저귀밴드/팬티',
       '아이정보건너뛰기', '구매시월령_bin', '총주문횟수', '구매일_y'],
      dtype='object')

In [23]:
df=df[['주문번호', '배송완료일', '배송시작일', '구매일_x', '구매시월령', '고객번호', '구매금액', '결제금액',
       '물품대분류', '상품명', '성별', '결혼유무', '거주지역', '연령', '결제등록카드', '유입경로', '자녀여부',
       '물품판매량', '제품단가', '주문일로부터배송완료시간', '택배이동기간', '기저귀단계', '남아여아공용', '브랜드',
       '원플원', '핫딜', '쿠폰적용가강조', '무료배송강조', '증정강조', '예약구매', '기저귀밴드/팬티',
       '아이정보건너뛰기', '구매시월령_bin',  '총주문횟수']]


df.columns = ['주문번호', '배송완료일', '배송시작일', '구매일', '구매시월령', '고객번호', '구매금액', '결제금액',
       '물품대분류', '상품명', '성별', '결혼유무', '거주지역', '연령', '결제등록카드', '유입경로', '자녀여부',
       '물품판매량', '제품단가', '주문일로부터배송완료시간', '택배이동기간', '기저귀단계', '남아여아공용', '브랜드',
       '원플원', '핫딜', '쿠폰적용가강조', '무료배송강조', '증정강조', '예약구매', '기저귀밴드/팬티',
       '아이정보건너뛰기', '구매시월령_bin',  '총주문횟수']

In [24]:
df['마지막구매일'] = df.groupby('고객번호')['구매일'].transform('max')

df['고객휴면일수'] = (df['배송완료일'].max()-df['마지막구매일']).dt.days



# 등급 부여 함수
def 등급부여(금액):
    if 금액 >= 300000:
        return '스타'
    elif 금액 >= 150000:
        return '다이아몬드'
    elif 금액 >= 80000:
        return '플래티넘'
    elif 금액 >= 30000:
        return '골드'
    elif 금액 >= 1:
        return '실버'
    else:
        return '브론즈'

# 1) 고객별 마지막 주문일 계산
last_order = df.groupby('고객번호')['구매일'] \
               .max() \
               .reset_index(name='최종구매일')
# 2) 원본 df에 붙이기
df2 = df.merge(last_order, on='고객번호', how='left')

# 3) "3개월 이내" 여부 플래그
df2['최근3개월'] = df2['구매일'] >= df2['최종구매일'] - pd.DateOffset(months=3)

# 4) 고객별 최근 3개월 합산
고객별_3개월합 = (
    df2[df2['최근3개월']]
    .groupby('고객번호')['결제금액']
    .sum()
    .reset_index(name='최근3개월구매금액')
)

# 5) 원본 df에 합산 결과 병합
df = df.merge(고객별_3개월합, on='고객번호', how='left')

df['고객등급']=df['최근3개월구매금액'].apply(lambda x : 등급부여(x))

# 고객 등급에 따른 혜택 컬럼 생성
df['등급별혜택_상품쿠폰'] = np.where(df['고객등급']=='스타','20%쿠폰1장',
         np.where(df['고객등급']=='다이아몬드','20%쿠폰1장',
                  np.where(df['고객등급']=='플래티넘','15%쿠폰1장',
                           np.where(df['고객등급']=='골드','15%쿠폰1장',
                                    np.where(df['고객등급']=='실버','15%쿠폰1장',
                                             np.where(df['고객등급']=='브론즈','이달의쿠폰팩',np.nan))))))

df['등급별혜택_쇼핑지원금'] = np.where(df['고객등급']=='스타','2천원 4장',
         np.where(df['고객등급']=='다이아몬드','2천원 3장',
                  np.where(df['고객등급']=='플래티넘','2천원 2징',
                           np.where(df['고객등급']=='골드','2천원 1장',
                                    np.where(df['고객등급']=='실버','1천원 1장',
                                             np.where(df['고객등급']=='브론즈','X',np.nan))))))

df['등급별혜택_무료배송쿠폰'] = np.where(df['고객등급']=='스타','배송비 무료 5회',
         np.where(df['고객등급']=='다이아몬드','배송비 무료 4회',
                  np.where(df['고객등급']=='플래티넘','배송비 무료 3회',
                           np.where(df['고객등급']=='골드','배송비 무료 2회',
                                    np.where(df['고객등급']=='실버','X',
                                             np.where(df['고객등급']=='브론즈','X',np.nan))))))

df['거주지역'] = df['거주지역'].replace({'강원':'강원','경기':'경기','경남':'경상도','경북':'경상도','광주':'광주'
                  ,'대구':'대구','대전':'대전','부산':'부산','서울':'서울','인천':'인천'
                   ,'전남':'전라도','전북':'전라도','제주':'제주','충남':'충청도','충북':'충청도'
                })

df['태블로용_거주지역'] = df['거주지역'].replace({'강원':'강원','경기':'경기','경남':'경상남도','경북':'경상북도','광주':'광주'
                  ,'대구':'대구','대전':'대전','부산':'부산','서울':'서울','인천':'인천'
                   ,'전남':'전라남도','전북':'전라북도','제주':'제주','충남':'충청남도','충북':'충청북도'
                })

# 이탈 등급
# 마지막 구매 1년 이상 -> 이탈
# 6개월 ~ 1년 -> 휴면 고객
# 3개월 ~ 6개월 -> 휴면예비군
# ~ 3개월 -> 활성화고객
df['이탈등급'] = np.where(df['고객휴면일수']>30*12, '이탈'
         ,np.where(df['고객휴면일수']>30*6, '휴면 고객'
                   ,np.where(df['고객휴면일수']>30*3, '휴면예비군','활성화고객')))

로켓배송 = '2022-05-23'
df['당일출고서비스유무'] = np.where(df['구매일'] <= 로켓배송, '당일출고 서비스 시작 전', '당일출고 서비스 시작 후')

df['배송시작일_year'] = df['배송시작일'].dt.year
df['배송시작일_month'] = df['배송시작일'].dt.month
df['배송시작일_day'] = df['배송시작일'].dt.day

# 만약 문자열로 되어 있다면
df['주문일로부터배송완료시간'] = df['주문일로부터배송완료시간'].astype(int)

In [29]:
df = df.drop(columns='상품명')
df = df.reset_index(drop=True)
df.shape

(185354, 46)

In [ ]:
df.to_csv('전처리완료.csv', index=False, encoding='CP949')